In [48]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,f1_score
from collections import Counter
from pylab import *
import nltk
import warnings
warnings.filterwarnings('ignore')


lemmatizer = WordNetLemmatizer()

In [4]:
review_data = pd.read_json('data/reviews_Musical_Instruments_5.json', lines=True)
print(review_data.shape)
review_data.head()

(10261, 9)


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [6]:
review_data['overall'].value_counts()

5    6938
4    2084
3     772
2     250
1     217
Name: overall, dtype: int64

In [8]:
# clearning text
review_data['cleaned_review_text'] = review_data['reviewText'].apply(\
lambda x : ' '.join([lemmatizer.lemmatize(word.lower()) \
    for word in word_tokenize(re.sub(r'([^\s\w]|_)+', ' ', str(x)))]))

In [12]:
review_data[['reviewText','cleaned_review_text']].head()

,reviewText,cleaned_review_text
0,"Not much to write about here, but it does exac...",not much to write about here but it doe exactl...
1,The product does exactly as it should and is q...,the product doe exactly a it should and is qui...
2,The primary job of this device is to block the...,the primary job of this device is to block the...
3,Nice windscreen protects my MXL mic and preven...,nice windscreen protects my mxl mic and preven...
4,This pop filter is great. It looks and perform...,this pop filter is great it look and performs ...


In [20]:
tfidf_model = TfidfVectorizer(max_features=200)
tfidf = tfidf_model.fit_transform(review_data['cleaned_review_text']).todense()
tfidf_df = pd.DataFrame(tfidf,columns=tfidf_model.get_feature_names())
tfidf_df.head()

,34,about,acoustic,after,all,also,always,am,amp,an,...,which,while,will,with,without,work,would,year,you,your
0,0.0,0.181565,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.152732,0.000000,0.0,0.000000,0.00000
1,0.0,0.000000,0.0,0.138705,0.0,0.0,0.0,0.0,0.0,0.215048,...,0.0,0.000000,0.105732,0.072374,0.0,0.098105,0.000000,0.0,0.077020,0.00000
2,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.190905,0.000000,0.191615,0.0,0.000000,0.147183,0.0,0.101957,0.14294
3,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000
4,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.248106,0.000000,0.0,0.000000,0.000000,0.0,0.361463,0.00000


In [43]:
# lets make a target feature which indicates that a review is postive (1) or negative (0)
# target = 1,if overall = 5. else its negatve

review_data['target'] = np.where(review_data['overall']>4,1,0)
review_data['target'].value_counts()

1    6938
0    3323
Name: target, dtype: int64

# Logisitic regression

In [44]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(tfidf_df,review_data['target'])
predicted_labels = logreg.predict(tfidf_df)

In [45]:
logreg.predict_proba(tfidf_df)[:,1]

array([0.48870299, 0.60974545, 0.54251376, ..., 0.7264981 , 0.52227241,
       0.40689435])

In [46]:
review_data['predicted_labels'] = predicted_labels

In [47]:
pd.crosstab(review_data['target'], review_data['predicted_labels'])

predicted_labels,0,1
target,,
0,1319,2004
1,684,6254


In [49]:
accuracy_score(review_data['target'],predicted_labels)

0.7380372283403177

In [50]:
f1_score(review_data['target'],predicted_labels)

0.8231113450908134

# Naive Bayes Classifier

In [51]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(tfidf_df,review_data['target'])
predicted_labels = nb.predict(tfidf_df)

In [52]:
review_data['predicted_labels'] = predicted_labels

In [53]:
pd.crosstab(review_data['target'], review_data['predicted_labels'])

predicted_labels,0,1
target,,
0,2185,1138
1,2368,4570


In [54]:
accuracy_score(review_data['target'],predicted_labels)

0.6583179027385245

In [55]:
f1_score(review_data['target'],predicted_labels)

0.7227581844061363

# K nearest Neighbor

In [61]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(tfidf_df,review_data['target'])
predicted_labels = knn.predict(tfidf_df)

In [62]:
review_data['predicted_labels'] = predicted_labels

In [63]:
pd.crosstab(review_data['target'], review_data['predicted_labels'])

predicted_labels,0,1
target,,
0,2089,1234
1,762,6176


In [64]:
accuracy_score(review_data['target'],predicted_labels)

0.8054770490205633

In [65]:
f1_score(review_data['target'],predicted_labels)

0.8608865347086703

# Regression

- # Linear Regression

In [66]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(tfidf_df,review_data['overall'])

LinearRegression()

In [67]:
review_data['predicted_score_from_linear_regression'] = linreg.predict(tfidf_df)

In [69]:
review_data[['overall','predicted_score_from_linear_regression']].head(10)

,overall,predicted_score_from_linear_regression
0,5,4.208884
1,5,3.871790
2,5,4.300055
3,5,4.253997
4,5,4.600466
5,5,4.971764
6,5,4.320081
7,3,4.203550
8,5,4.836814
9,5,4.882585
